In [1]:
#| default_exp tbats

# TBATS model 

In [2]:
#| export
import os
import numpy as np 
from numba import njit 
from scipy.optimize import minimize_scalar
import statsmodels.api as sm 
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

In [3]:
#| export 
# Global variables 
NOGIL = os.environ.get('NUMBA_RELEASE_GIL', 'False').lower() in ['true']
CACHE = os.environ.get('NUMBA_CACHE', 'False').lower() in ['true']

## Load data 

In [4]:
#| hide 
# AirPassengers 
from statsforecast.utils import AirPassengers as ap 

# USAccDeaths
deaths = np.array([9007,  8106,  8928,  9137, 10017, 10826, 11317, 10744,  9713,  9938,  9161,  8927,
7750,  6981,  8038,  8422,  8714,  9512, 10120,  9823,  8743,  9129,  8710,  8680,
8162,  7306,  8124,  7870,  9387,  9556, 10093,  9620,  8285,  8466,  8160,  8034,
7717,  7461,  7767,  7925,  8623,  8945, 10078,  9179,  8037,  8488,  7874,  8647,
7792,  6957,  7726,  8106,  8890,  9299, 10625,  9302,  8314,  8850,  8265,  8796,
7836,  6892,  7791,  8192,  9115,  9434, 10484,  9827,  9110,  9070,  8633,  9240])

# Electricity consumption
import pandas as pd 
elec_df = pd.read_csv('/Users/marianamenchero/Documents/tbats/data/PJMW_hourly.csv')
elec = elec_df['PJMW_MW']
elec = elec[0:(24*7*3)]
elec = np.array(elec)

# Calls 
calls_df = pd.read_csv('/Users/marianamenchero/Documents/tbats/data/calls_data.csv')
calls = calls_df['calls']
calls = calls[-2001:]
calls = np.array(calls)

# M4 data 
train = pd.read_csv('/Users/marianamenchero/Documents/tbats/data/Hourly-train.csv')
uid = 'H50'
m4_df = train.loc[train['unique_id'] == uid]
y = m4_df['y'].values
seasonal_periods = np.array([24])

/Users/marianamenchero/miniforge3/envs/nixtla-stats/lib/python3.10/site-packages/statsforecast/core.py:25: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Box-Cox Transformation

In [5]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def guer_cv(lam, x, season_length): 
    
   """Minimize this funtion to find the optimal parameter for the Box-Cox transformation."""
   period = np.round(np.max(season_length)) 
   n = len(x) 
   nyears = int(np.floor(n/period))
   nobs = np.floor(nyears*period)
   m = int(n-nobs)
   xmat = x[m:n].reshape((nyears, period))

   xmean = np.full(xmat.shape[0], fill_value = np.nan)
   for k in range(xmat.shape[0]): 
      xmean[k] = np.nanmean(xmat[k])

   xsd = np.full(xmat.shape[0], fill_value = np.nan)
   for k in range(xmat.shape[0]): 
      vals = xmat[k]
      svar = (vals-np.nanmean(vals))**2 
      svar = np.sum(svar)/len(svar) 
      xsd[k] = np.sqrt(svar) 

   xrat = xsd/(xmean**(1-lam))

   sd = (xrat-np.nanmean(xrat))**2 
   sd = np.nansum(sd)/(len(sd)-1)
   sd = np.sqrt(sd)
   
   return sd/np.nanmean(xrat)

In [6]:
#| exporti
def guerrero(x, season_length, lower=0, upper=1): 
    
    """Finds optimal paramater for Box-Cox transformation using Guerrero's method"""
    
    if np.any(x < 0): 
        raise ValueError("Guerrero's method for selecting a Box-Cox parameter (w) is for strictly positive data")
    
    max_freq = np.max(season_length)
    if len(x) <= 2*max_freq: 
        res = 1
    else: 
        w = 0 # initial guess 
        opt = minimize_scalar(guer_cv, w, args = (x, season_length), method = 'bounded', bounds = (lower, upper))
        res = opt.x

    return res

In [7]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def BoxCox(y, BoxCox_lambda): 
    
    """Applies Box-Cox transformation with parameter BoxCox_lambda"""

    if BoxCox_lambda == 0: 
        z = np.log(y)
    else: 
        z = np.sign(y)*((np.abs(y)**BoxCox_lambda)-1)
        z = z/BoxCox_lambda
        
    return z

In [8]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def InverseBoxCox(z, BoxCox_lambda): 
    
    """Inverts Box-Cox transformation with parameter BoxCox_lambda"""

    if BoxCox_lambda == 0: 
        y = np.exp(z) 
    else: 
        sign = np.sign(BoxCox_lambda*z+1)
        y = np.abs(BoxCox_lambda*z+1)**(1/BoxCox_lambda)
        y = sign*y 
        
    return y 

## Functions 

### find_harmonics

In [9]:
#| exporti
def find_harmonics(y, seasonal_periods, k):

    # Compute a 2 x m moving average to estimate the trend 
    m = seasonal_periods[k]
    window_size = 2 * m
    data = pd.DataFrame({'value': y})
    f_t = data['value'].rolling(window = window_size, center = True).mean()
    
    # Obtain an approximation of seasonal component using z_t = y_t - f_t 
    data['f_t'] = f_t
    data['z_t'] = data['value']-data['f_t']

    # Drop missing values (due to the moving average)
    data.dropna(inplace = True) 

    if data.empty:
        return 1

    # Approximate the seasonal component using trigonometric terms
    t = np.arange(len(data))
    if m % 2 == 0: 
        max_harmonics = int(m/2)
    else: 
        max_harmonics = int((m-1)/2)

    max_harmonics = min(max_harmonics, len(data))
    
    aic = np.inf
    num_harmonics = 0
    for h in range(1,max_harmonics+1): 
        columns = []
        for i in range(1, h+1):
            data[f'cos_{i}'] = np.cos(2*np.pi*i*t/m)
            data[f'sin_{i}'] = np.sin(2*np.pi*i*t/m)
            columns.extend([f'cos_{i}', f'sin_{i}'])
        
        # Perform regression to estimate the coefficients
        X = data[columns]
        y = data['z_t']
        model = sm.OLS(y, X).fit()
        new_aic = model.aic 

        if new_aic < aic: 
            aic = new_aic
            num_harmonics = h  
        else: 
            break

    if num_harmonics == 0: 
        num_harmonics += 1 
    
    return num_harmonics

### initial_parameters 

In [10]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def initial_parameters(seasonal_periods, k_vector, use_trend, use_damped_trend, ar_coeffs, ma_coeffs): 

    alpha = 0.09 

    if use_trend: 
        adj_beta = 1 
        beta = 0.05
        b = 0 
        if use_damped_trend: 
            phi = 0.999 
        else: 
            phi = 1
    else: 
        adj_beta = 0 
        beta = None 
        b = None 
        phi = None

    if seasonal_periods is not None: 
        gamma_one_v = np.zeros(len(k_vector))
        gamma_two_v = np.zeros(len(k_vector))
        s_vector = np.zeros(2*np.sum(k_vector)) 
    # else:  # for this to happen, seasonal_periods must be None, which is not possible in StatsForecast
    #     gamma_one_v = None 
    #     gamma_two_v = None 
    #     s_vector = None 

    if ar_coeffs is not None: 
        d_vector = np.zeros(len(ar_coeffs))
    else: 
        d_vector = None
    if ma_coeffs is not None:
        epsilon_vector = np.zeros(len(ma_coeffs))  
    else: 
        epsilon_vector = None 

    return alpha, adj_beta, beta, b, phi, gamma_one_v, gamma_two_v, s_vector, d_vector, epsilon_vector 

### makeXMatrix

In [11]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def makeXMatrix(b, s_vector, d_vector, epsilon_vector): 

    # x = (l_t, b_t, s_vector, d_vector, epsilon_vector)
    x = np.array([0.0]) 
    if b is not None: 
        x = np.append(x, b)
    if s_vector is not None: 
        x = np.concatenate((x, s_vector))
    if d_vector is not None: 
        x = np.concatenate((x, d_vector)) 
    if epsilon_vector is not None: 
        x = np.concatenate((x, epsilon_vector))
    
    return x 

### findPQ

In [12]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def findPQ(ar_coeffs, ma_coeffs): 
    if ar_coeffs is not None: 
        p = len(ar_coeffs) 
    else: 
        p = 0 
    if ma_coeffs is not None: 
        q = len(ma_coeffs) 
    else: 
        q = 0 
    return p, q

### makeTBATSWMatrix

In [13]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def makeTBATSWMatrix(phi, k_vector, ar_coeffs, ma_coeffs, tau):
    
    # w_transpose = (1, phi, a, varphi, theta)
    p, q = findPQ(ar_coeffs, ma_coeffs)
    adj_phi = 0 
    numCols = 1

    if phi is not None: 
        adj_phi += 1
        numCols += 1

    numCols += tau+p+q

    w_transpose = np.zeros((1,numCols))

    position = adj_phi
    for s in range(len(k_vector)): 
        w_transpose[0, position+1:position+k_vector[s]+1] = 1
        position += 2*k_vector[s]

    w_transpose[0,0] = 1 

    if phi is not None: # equivalent to asking adj_phi == 1
        w_transpose[0,1] = phi

    if ar_coeffs is not None: 
        w_transpose[0, adj_phi+tau+1:adj_phi+tau+p+1] = ar_coeffs

    if ma_coeffs is not None: 
        w_transpose[0, adj_phi+tau+p+1:adj_phi+tau+p+q+1] = ma_coeffs

    return w_transpose

### makeTBATSGMatrix 

In [14]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def makeTBATSGMatrix(k_vector, alpha, adj_beta, beta, gamma_one_v, gamma_two_v, p, q, tau):
    
    # g = (alpha, beta, gamma_bold, 1, 0_{p-1}, 1, 0_{q-1})
    g = np.zeros((1+adj_beta+2*np.sum(k_vector)+p+q,1))

    g[0,0] = alpha 

    if beta is not None: 
        g[1,0] = beta 

    gamma_bold = np.ones((1, 2*sum(k_vector)))
    endPos = 0 
    for i in range(len(k_vector)): 
        for j in range(endPos, endPos+k_vector[i]): 
            gamma_bold[0,j] = gamma_one_v[i]
        for k in range(endPos+k_vector[i], endPos+2*k_vector[i]): 
            gamma_bold[0,k] = gamma_two_v[i]
        endPos += 2*k_vector[i]

    g[1+adj_beta:1+adj_beta+gamma_bold.shape[1],0] = gamma_bold

    if p != 0: 
        g[1+adj_beta+tau,0] = 1 
    if q != 0: 
        g[1+adj_beta+tau+p,0] = 1

    return g

## TBATS model 

In [15]:
y = elec
seasonal_periods = np.array([24, 24*7])

In [16]:
use_boxcox = True
bc_lower_bound = 0
bc_upper_bound = 1
use_trend = True
use_damped_trend = True
use_arma_errors = True

In [17]:
# def model_generator(y, seasonal_periods, use_boxcox, bc_lower_bound, bc_upper_bound, use_trend, use_damped_trend, use_arma_errors, k_vector): 
#     return 42

In [18]:
# def tbats_model(y, seasonal_periods, use_boxcox, bc_lower_bound, bc_upper_bound, use_trend, use_damped_trend, use_arma_errors): 
#     return 42 

In [19]:
# tbats_model 

# 1. Sort seasonal periods 
seasonal_periods = np.sort(seasonal_periods)

# 2. Check if there are missing values 
indices = np.where(np.isnan(y))[0]
if len(indices) > 0: 
    max_index = indices[-1]
    y = y[max_index+1:len(y)]

# 3. Check if there are negative values
if np.any(y < 0): 
        use_boxcox = False 

# 4. Check if there is a trend
adf = adfuller(y, regression = 'ct')
if adf[1] <= 0.05: 
    print('The time series is trend-stationary')
    use_trend = False
    use_damped_trend = False

# 5. Choose the number of harmonics 
k_vector = np.zeros(len(seasonal_periods)).astype(int)

for k in range(len(seasonal_periods)): 
    k_vector[k] = find_harmonics(y, seasonal_periods, k)

print('k_vector:'+str(k_vector))

# 6. Initial Box-Cox transformation (if required)
if use_boxcox: 
      BoxCox_lambda = 0 # This is the initial value used in the thesis 
      y_trans = BoxCox(y, BoxCox_lambda)
else: 
      BoxCox_lambda = None
      y_trans = y 

k_vector:[5 1]


In [20]:
# 7. Create first model - No ARMA errors 
ar_coeffs = np.array([1.51, -0.55])
ma_coeffs = np.array([-0.314])

p, q = findPQ(ar_coeffs, ma_coeffs)

# ar_coeffs = None 
# ma_coeffs = None 

# select starting params 
alpha, adj_beta, beta, b, phi, gamma_one_v, gamma_two_v, s_vector, d_vector, epsilon_vector = initial_parameters(seasonal_periods, k_vector, use_trend, use_damped_trend, ar_coeffs, ma_coeffs)

# seed states 
x_nought = makeXMatrix(b, s_vector, d_vector, epsilon_vector)

# tau 
tau = 2*np.sum(k_vector)

# Create w_transpose, g, and F matrices 
w_transpose = makeTBATSWMatrix(phi, k_vector, ar_coeffs, ma_coeffs, tau)

g = makeTBATSGMatrix(k_vector, alpha, adj_beta, beta, gamma_one_v, gamma_two_v, p, q, tau)

In [ ]:
# 8. Create second model - ARMA errors 

# 9. Select model with the lowest AIC (with and without ARMA errors)

# 10. Repeat with all possible classes of models and select the model with the lowest AIC. This will be the final model. 